In [1]:
from finetuning_utils import *
import torch.nn as nn

Using cpu as DEVICE


In [2]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
weights_dir = config['data']['model_weights']

### Load model

In [3]:
model_name = 'loveda_DeepLabV3_r101_epochs15_lr0-01'
model = torch.load(weights_dir + '/own_training/' + model_name + '/' + model_name + '_epoch12.pth.tar', 
                map_location=torch.device('cpu'))

### Get Loaders

In [4]:
train_loader, test_loader = get_munich_tuning_loaders(batch_size=2)

Indexing files in orthophotos/labeling_subset... 
Found: 	 150 Images 
	 83 Mask
Length of all files: 83
Length of train (66) and test (17): 83
Length of train loader: 33; Length of test loader: 9 with batch size 2


### Evaluation

In [5]:
calculate_accuracy(model, test_loader, 6)


Evaluating Accuracy on Test Set...


100%|██████████| 9/9 [00:57<00:00,  6.40s/it]


,Metric,Accuracy (%)
0,ignore,No instances in the test set
1,impervious,67.16
2,building,80.36
3,low vegetation,76.77
4,water,91.96
5,trees,85.6
6,Mean Accuracy,80.37
7,Average Accuracy,78.082657
